In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 四维数据
# 其实要找到主成分就是找到方差最大
# 可以去求协方差矩阵的最大特征值对应的特征向量
# 但这样不符合机器学习
# 所以还是通过梯度上升/下降来学习
# 先从读数据集开始，跟前面一次作业差不多
# 因为不需要做K折一些的，而且是用全部数据所以没必要打乱
data = pd.read_csv('iris.csv', header=0, index_col=0)

# 默认的数据集index是从1开始的，非常不适合程序员啊，所以改一下
data.index = data.index-1

# 然后先标准化，得在不同的列上处理（不同特征不能混在一起）
# 这个地方，有一个标准化和归一化的争议
# 首先，为了满足假设需要的是均值为0，方差为1
# 然后，英语一个是Standardization，一个是Normalization
# 网上对这两个概念弄反的很多
# 互联网发展起来后真烦
# 尤其是那些乱用爬虫的，还有就是那些发现问题不积极讨论的
# 然后因为老师说最好不要用ML库，我就自己手写一个
def standarlize(data):
    # 保护原有数据
    data_ = data.copy()
    data_ = data_.iloc[:, :-1]
    # 不要遍历了最后一行，我就因为这个原因一直说有非数值的
    for index, row in data_.iteritems():
        data_.iloc[:][index] = (row - row.mean()) / row.std()
    return data_

data_stand = standarlize(data)
# 通过DataFrame的操作直接获得第二问的数据
data_Setorsa = data.loc[data['Species']=='setosa']
# 同样提前标准化一下
data_stand_Setorsa = standarlize(data_Setorsa)

In [3]:
# 接下来开始寻找第一主成分
# 整体分析就是让投影和最大
# 那么关于v利用梯度上升即可
# 概念啥的见笔记
# 突然意识到用pd并不是很好处理纯数值数据，引入并转换为np
data_stand = np.array(data_stand)
data_stand_Setorsa = np.array(data_stand_Setorsa)

# 定义优化目标
def f(v, X):
    return np.sum((X.dot(v)**2)) /len(X)

# 定义梯度，平方的2拿下来会被学习率吸收
def df(v, X):
    return X.T.dot(X.dot(v)) / len(X)

# 需要将梯度更新后的向量再更新为单位向量
def norm(v):
    return v / np.linalg.norm(v)

In [4]:
# 梯度上升，两个结束条件，一个收敛标志
def gradient_ascent(X, lr=0.1, epsilon=1e-8):
    v = norm(np.random.random(X.shape[1]))
    print("初始化单位向量为", v)
    flag = False
    i = 0
    while i < 10000:
        v_ = v.copy()
        gradient = df(v, X)
        v += lr * gradient
        v = norm(v)
        i+=1
        if abs(f(v, X) - f(v_, X)) < epsilon:
            flag = True
            break
        
    if flag:
        print("已经收敛，迭代次数为",i, "。最终的1st主成分为", v)
        print("重构误差为：", error(X, v))
    else:
        print("没有收敛")

In [5]:
# 计算重构误差，因为移动到了原点附近
# 所以就是计算所有点的模减去v的模开根号后的和
# 想了想好像先计算向量，然后再计算距离和也可以
def error(X, v):
    v = v.reshape(1, -1)
    err = X - (X.dot(v.T)).dot(v)
    err_d = np.sum(err**2, axis=1)
    err_sum = np.sum(np.sqrt(err_d))
    return err_sum

In [6]:
gradient_ascent(data_stand)

初始化单位向量为 [0.51537887 0.77533153 0.24154966 0.27367754]
已经收敛，迭代次数为 61 。最终的1st主成分为 [ 0.52110187 -0.2692594   0.58041544  0.56486293]
重构误差为： 131.81257698530374


In [7]:
gradient_ascent(data_stand_Setorsa)

初始化单位向量为 [0.09545448 0.69743357 0.38402957 0.59749155]
已经收敛，迭代次数为 81 。最终的1st主成分为 [0.60434354 0.5755267  0.37555997 0.40310373]
重构误差为： 60.89127796815511
